In [1]:
%pip install ultralytics onnxruntime opencv-python numpy matplotlib torch torchvision torchaudio

  Using cached torchaudio-2.6.0-cp313-cp313-win_amd64.whl.metadata (6.7 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.2/11.3 MB 3.3 MB/s eta 0:00:04
   - -------------------------------------- 0.6/11.3 MB 6.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.3 MB 9.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.3 MB 14.2 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.3 MB 15.1 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.3 MB 15.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.3/11.3 MB 13.5 MB/s eta 0:00:01
   ------------------ --------------------- 5.2/11.3 MB 15.5 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.3 MB 15.8 MB/s eta 0:00:01
   ------------------------ --------------- 6.9/11.3 MB 14.9 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.3 MB 15.5 MB/s eta 0:00:01

Could not find platform independent libraries <prefix>

[notice] A new release of pip is available: 23.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Define save path
yolov12m_model_url = r"D:\Courses\ML-based-suture-pad-analysis\ml-models\models"

In [11]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# Load the best-trained model (Update the path once YOLOv12 is available)
model_path = "yolo12m.pt"  # Update with YOLOv12 model when available

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(model_path).to(device)

print(f"Model loaded on {device}")

os.makedirs(yolov12m_model_url, exist_ok=True)  # Ensure the directory exists

# Save the model
save_path = os.path.join(yolov12m_model_url, "yolo12m_saved.pt")
model.save(save_path)

print(f"Model saved at {save_path}")

Model loaded on cpu
Model saved at D:\Courses\ML-based-suture-pad-analysis\ml-models\models\yolo12m_saved.pt


In [12]:
class_names = {
    0: "incision_bent",
    1: "incision_good",
    2: "incision_perfect",
    3: "knot_good",
    4: "knot_loose",
    5: "knot_perfect",
    6: "knot_tight",
    7: "suture_good",
    8: "suture_loose",
    9: "suture_perfect",
    10: "suture_tight",
    11: "tail_end",
    12: "tail_top"
}

In [4]:
def run_yolov12_inference(image_path, conf_threshold=0.3, iou_threshold=0.5):
    """
    Run YOLOv12 on an image and return detections.
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image {image_path}")
        return None

    # Perform detection
    results = model.predict(source=image_path, conf=conf_threshold, iou=iou_threshold, device=device)

    # Extract detections
    detections = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            conf = float(box.conf)
            bbox = box.xyxy.cpu().numpy().tolist()[0]  # [x1, y1, x2, y2]
            detections.append((class_id, conf, bbox))

    return detections, img

In [13]:
def visualize_detections(image_path, detections):
    """
    Draw bounding boxes on the image.
    """
    img = cv2.imread(image_path)
    for class_id, conf, bbox in detections:
        x1, y1, x2, y2 = map(int, bbox)
        label = f"{class_names[class_id]}: {conf:.2f}"
        color = (0, 255, 0)  # Green for detected objects
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    plt.figure(figsize=(8, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

In [14]:
def crop_detected_objects(image_path, detections, save_dir="cropped_sutures"):
    """
    Crop and save detected objects.
    """
    os.makedirs(save_dir, exist_ok=True)
    img = cv2.imread(image_path)

    for idx, (class_id, conf, bbox) in enumerate(detections):
        x1, y1, x2, y2 = map(int, bbox)
        cropped_img = img[y1:y2, x1:x2]

        if cropped_img.size == 0:
            continue

        cropped_filename = os.path.join(save_dir, f"{class_names[class_id]}_{idx}.jpg")
        cv2.imwrite(cropped_filename, cropped_img)

    print(f"Cropped images saved in {save_dir}")

In [ ]:
image_path = "sample_suture.jpg"  # Replace with your image
detections, img = run_yolov12_inference(image_path)

if detections:
    visualize_detections(image_path, detections)
    crop_detected_objects(image_path, detections)


In [16]:
def convert_yolov12_to_onnx(model_path="yolov12m.pt", output_onnx="yolov12.onnx"):
    """
    Convert YOLOv12 PyTorch model to ONNX.
    """
    model = YOLO(model_path)
    model.export(format="onnx", dynamic=True, device=device)
    print(f"Model converted to {output_onnx}")

In [20]:
%pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" pyproject.toml-based projects


     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
     --------------------------------------- 0.0/12.2 MB 385.2 kB/s eta 0:00:32
     --------------------------------------- 0.1/12.2 MB 660.4 kB/s eta 0:00:19
      --------------------------------------- 0.3/12.2 MB 2.0 MB/s eta 0:00:06
     ---- ----------------------------------- 1.3/12.2 MB 7.0 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.2 MB 14.1 MB/s eta 0:00:01
     ----------------- ---------------------- 5.4/12.2 MB 19.0 MB/s eta 0:00:01
     ------------------------- -------------- 7.9/12.2 MB 24.0 MB/s eta 0:00:01
     -------------------------------- ------ 10.0/12.2 MB 26.5 MB/s eta 0:00:01
     --------------------------------- ----- 10.5/12.2 MB 43.7 MB/s eta 0:00:01
     ------------------------------------ -- 11.4/12.2 MB 36.0 MB/s eta 0:00:01
     --------------------------------------- 12.2/12.2 MB 38.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Instal

Could not find platform independent libraries <prefix>
ERROR: Could not find a version that satisfies the requirement pyproject.toml-based (from versions: none)
ERROR: No matching distribution found for pyproject.toml-based

[notice] A new release of pip is available: 23.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# Run conversion
convert_yolov12_to_onnx(model_path=save_path, output_onnx="yolov12.onnx")

Ultralytics 8.3.78  Python-3.13.1 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
YOLOv12m summary (fused): 169 layers, 20,166,592 parameters, 0 gradients, 67.5 GFLOPs

PyTorch: starting from 'D:\Courses\ML-based-suture-pad-analysis\ml-models\models\yolo12m_saved.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (39.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim" ' returned non-zero exit status 1.
ONNX: export failure  132.2s: No module named 'onnx'


ModuleNotFoundError: No module named 'onnx'

In [ ]:
import onnxruntime as ort

onnx_model = "yolov12.onnx"
session = ort.InferenceSession(onnx_model, providers=["CUDAExecutionProvider"] if torch.cuda.is_available() else ["CPUExecutionProvider"])

def run_onnx_inference(image_path):
    """
    Run inference using ONNX Runtime for faster processing.
    """
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (640, 640))
    img_resized = np.expand_dims(img_resized.transpose(2, 0, 1), axis=0).astype(np.float32)

    inputs = {session.get_inputs()[0].name: img_resized}
    outputs = session.run(None, inputs)

    return outputs

# Run ONNX inference
onnx_results = run_onnx_inference(image_path)
print("ONNX Inference Complete")